In [1]:
import pandas as pd
import numpy as np
import random
from datetime import timedelta
from datetime import datetime



## Load Task Duration Config
For each task type that could be generated we need an average duration and a maximum duration.

In [2]:
#Load task duration dataframe
#task_duration_df = pd.read_csv('task_duration_config.csv')
#task_duration_df.set_index('Task', inplace=True)

data = [['New Client Onboarding Request', 0.5,1.5], 
        ['Review Documents', 0.25,  2.5], 
        ['Automated Scoreboarding', 0.1, 0.15],
        ['Manual Scoreboarding', 1.0, 3.0],
        ['Update Backend Systems', 0.25, 0.5],
        ['Notification Review Request Completed', 0.1, 0.15]
       ]

task_duration_df = pd.DataFrame(data, columns=['Task', 'Avg', 'Max'])
task_duration_df.set_index('Task', inplace=True)

In [3]:

#task_duration_df = pd.read_csv('task_duration_config.csv')
#task_duration_df.set_index('Task', inplace=True)

In [4]:
task_duration_df.head(10)

,Avg,Max
Task,,
New Client Onboarding Request,0.50,1.50
Review Documents,0.25,2.50
Automated Scoreboarding,0.10,0.15
Manual Scoreboarding,1.00,3.00
Update Backend Systems,0.25,0.50
Notification Review Request Completed,0.10,0.15


## Utility Functions

### Generate Process Instances
This function manages the creation of process instances for a specific process variant. A process variant is simply the list of tasks in execution order, including any loops. For example ['Wake Up', 'Breakfast', 'Work', 'Dinner' 'Sleep']

In [5]:
def generate_process_instances(process_variant, qty):

    global start_date_time
    global instance_counter
    task_list = []
    for x in range(0, qty):
        tasks = build_task_list(instance_counter, process_variant, start_date_time, task_duration_df)
        for task in tasks:
            task_list.append(task)

        # Increment the start time by 24 hour
        start_date_time = start_date_time + timedelta(hours=24)
        # Increment the process_id
        instance_counter = instance_counter +1
        
    return task_list

## Build Task List
The build_task_list function creates a list of tasks for a specific instance of a process. The sequence of tasks is defined in the variable called process_variant. Random task durations are calculated using the task_durations_df that contains the task name, the average duration and the max duration.  

This code generates a random duration by leveraging the numpy lognormal function that gives a random
number drawn from a log normal distribution. Look up a picture of log normal distributions and you'll see 
why this is useful for generating random durations based on a mean and std deviation.
most samples are near the average with a long tail stretching towards infinity.


In [50]:
def build_task_list(instance_id, process_variant, start_date_time, task_duration_df):
    instance_task_list = []
    rnd = np.random.default_rng()  # a random number generator

    # process_variant is a series
    for task_type in process_variant:
        # get the avg and max durations from the task_df dataframe using task_type as the key
        avg_dur = task_duration_df.loc[task_type, 'Avg']
        max_dur = task_duration_df.loc[task_type, 'Max']

        sigma = (max_dur - avg_dur) / max_dur  # std dev
        log_mean = np.log(avg_dur)  # can't pass the mean duration into lognormal until it has been logged itself 
        delta = rnd.lognormal(log_mean, sigma)  # get a random sample from a log normal distribution with a std dev
        end_time = start_date_time + timedelta(hours=delta)
        task = [instance_id, task_type, start_date_time, end_time]
        instance_task_list.append(task)

        # set the start time of the next tasks to be 5 mins after the end time of this task
        start_date_time = end_time + timedelta(minutes=5)

    return instance_task_list

## Shift Activity Start Time
This function shifts the activity start time for a specific task with a specific attribute set to a specific value. For example, to delay the start time of breakfast by 1hour "task_list, User, Gerry, Eat Breakfast, 1"

This function will apply the same timeshift to every subsequent task in the proccess instance. 

In [95]:
def shiftActivityStartTime(tasks, target_attribute, target_value, target_task, timeShift):

    # Logic requires we sort by process_id and start_date
    tasks = tasks.sort_values(['process_id', 'start_time'])
    tasks = tasks.reset_index(drop=True)
    process_being_modified = -1

    # iterate through the dataframe using the index value
    for x in tasks.index:
        row = tasks.loc[x]
        task_attribute = row[target_attribute]
        row_task = row['task']
        current_process = row['process_id']


        # Have we found the target activity
        if row_task == target_task and task_attribute == target_value:
            process_being_modified = current_process

        # Are we still processing the same process that we were when we found the target activity ?
        # If so we are moving all subsequent tasks back by the value of the timeshift parameter,
        # this means shifting the start time and the end time
        if current_process == process_being_modified:
            # shift the start time
            current_start = row['start_time']
            shifted_start = current_start + timedelta(hours=timeShift)
            tasks.loc[x, 'start_time'] = shifted_start
            
            # shift the end time
            current_end = row['end_time']
            shifted_end = current_end + timedelta(hours=timeShift)
            tasks.loc[x, 'end_time'] = shifted_end
            
    return tasks

In [128]:
def increaseDuration(tasks, target_attribute, target_value, target_task, timeShift):

    # Logic requires we sort by process_id and start_date
    tasks = tasks.sort_values(['process_id', 'start_time'])
    tasks = tasks.reset_index(drop=True)
    process_being_modified = -1

    # iterate through the dataframe using the index value
    for x in tasks.index:
        row = tasks.loc[x]
        task_attribute = row[target_attribute]
        row_task = row['task']
        current_process = row['process_id']
        processing_target_task = False

        # Have we found the target activity
        if row_task == target_task and task_attribute == target_value:
            process_being_modified = current_process
            processing_target_task = True

        # Are we still processing the same process that we were when we found the target activity ?
        # If so we are moving all subsequent tasks back by the value of the timeshift parameter,
        # this means shifting the start time and the end time
        if current_process == process_being_modified:
            
            if processing_target_task :
                # just increase the duration by increasing the end_time 
                current_end = row['end_time']
                shifted_end = current_end + timedelta(hours=timeShift)
                tasks.loc[x, 'end_time'] = shifted_end
            else:
                
                # increase start_time and end-time
                current_start = row['start_time']
                shifted_start = current_start + timedelta(hours=timeShift)
                tasks.loc[x, 'start_time'] = shifted_start
            
                # shift the end time
                current_end = row['end_time']
                shifted_end = current_end + timedelta(hours=timeShift)
                tasks.loc[x, 'end_time'] = shifted_end
            
    return tasks

## Initialise Data Generator

In [52]:
start_date_time = datetime(2017, 11, 28, 18, 00, 00)
instance_time_offset = 24
instance_counter = 1


## Create Happy Path Instances

In [53]:

qty = 900
happy_path = ["New Client Onboarding Request", 
               "Review Documents", 
               "Automated Scoreboarding",
               "Update Backend Systems", 
               "Notification Review Request Completed"]

happy_path_task_list = []
happy_path_task_list = generate_process_instances(happy_path, qty)
happy_path_task_list_df = pd.DataFrame(happy_path_task_list)

# Increment the start time by 24 hour
start_date_time = start_date_time + timedelta(hours=24)

In [54]:
happy_path_task_list_df.head()

,0,1,2,3
0,1,New Client Onboarding Request,2017-11-28 18:00:00.000000,2017-11-28 18:32:37.799670
1,1,Review Documents,2017-11-28 18:37:37.799670,2017-11-28 18:48:08.319336
2,1,Automated Scoreboarding,2017-11-28 18:53:08.319336,2017-11-28 18:59:32.546680
3,1,Update Backend Systems,2017-11-28 19:04:32.546680,2017-11-28 19:13:41.754157
4,1,Notification Review Request Completed,2017-11-28 19:18:41.754157,2017-11-28 19:26:44.284743


## Create Instances Requiring Manual Scoreboarding

In [55]:

qty = 80
manual_path = ["New Client Onboarding Request", 
               "Review Documents", 
               "Automated Scoreboarding",
               "Manual Scoreboarding",
               "Update Backend Systems", 
               "Notification Review Request Completed"]

manual_path_task_list = []
manual_path_task_list = generate_process_instances(manual_path, qty)
manual_path_task_list_df = pd.DataFrame(manual_path_task_list)

# Increment the start time by 24 hour
start_date_time = start_date_time + timedelta(hours=24)

In [56]:
manual_path_task_list_df.head()

,0,1,2,3
0,901,New Client Onboarding Request,2020-05-17 18:00:00.000000,2020-05-17 19:20:11.304127
1,901,Review Documents,2020-05-17 19:25:11.304127,2020-05-17 19:50:49.107159
2,901,Automated Scoreboarding,2020-05-17 19:55:49.107159,2020-05-17 20:02:52.693439
3,901,Manual Scoreboarding,2020-05-17 20:07:52.693439,2020-05-17 22:20:24.334532
4,901,Update Backend Systems,2020-05-17 22:25:24.334532,2020-05-17 22:51:37.388712


## Create Instance that loop

In [57]:
qty = 20
loop_path = ["New Client Onboarding Request", 
               "Review Documents", 
               "Automated Scoreboarding",
               "Manual Scoreboarding",
               "Review Documents",
               "Update Backend Systems", 
               "Notification Review Request Completed"]

loop_path_task_list = []
loop_path_task_list = generate_process_instances(loop_path, qty)
loop_path_task_list_df = pd.DataFrame(loop_path_task_list)

# Increment the start time by 24 hour
start_date_time = start_date_time + timedelta(hours=24)

In [73]:
# Can only concat two dataframes at a time
df = pd.concat([happy_path_task_list_df,manual_path_task_list_df], axis=0)
df = pd.concat([df,loop_path_task_list_df], axis=0)

In [74]:
df.columns = ['process_id', 'task', 'start_time', 'end_time']

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5120 entries, 0 to 139
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   process_id  5120 non-null   int64         
 1   task        5120 non-null   object        
 2   start_time  5120 non-null   datetime64[ns]
 3   end_time    5120 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 200.0+ KB


In [76]:
# Confirm how many processes
len(df["process_id"].unique())

1000

In [77]:
df.head(10)

,process_id,task,start_time,end_time
0,1,New Client Onboarding Request,2017-11-28 18:00:00.000000,2017-11-28 18:32:37.799670
1,1,Review Documents,2017-11-28 18:37:37.799670,2017-11-28 18:48:08.319336
2,1,Automated Scoreboarding,2017-11-28 18:53:08.319336,2017-11-28 18:59:32.546680
3,1,Update Backend Systems,2017-11-28 19:04:32.546680,2017-11-28 19:13:41.754157
4,1,Notification Review Request Completed,2017-11-28 19:18:41.754157,2017-11-28 19:26:44.284743
5,2,New Client Onboarding Request,2017-11-29 18:00:00.000000,2017-11-29 18:40:47.579016
6,2,Review Documents,2017-11-29 18:45:47.579016,2017-11-29 19:21:22.732782
7,2,Automated Scoreboarding,2017-11-29 19:26:22.732782,2017-11-29 19:33:29.563456
8,2,Update Backend Systems,2017-11-29 19:38:29.563456,2017-11-29 19:53:31.751123
9,2,Notification Review Request Completed,2017-11-29 19:58:31.751123,2017-11-29 20:07:52.740973


## Add Task Level Business Data : User

In [78]:

# Add a column for User
df["user"] = ""

In [79]:
def setRandomUser(row):
    match row["task"]:
        case "Review Documents":
            return random.choice(['Rod','Jane','Freddy'])
        case "New Client Onboarding Request":
            return random.choice(['Clive','Francis','Nick','Seb','Tom'])
        case "Manual Scoreboarding":
            return random.choice(['Sharon','Susan', 'Sam'])
        case "Update Backend Systems":
            return "RPA"
        case "Automated Scoreboarding":
            return "SYSTEM"
        case "Notification Review Request Completed":
            return "SYSTEM"
        case _:
            return row["user"]

In [80]:
df["user"] = df.apply(setRandomUser, axis=1)

In [81]:
df.head(20)

,process_id,task,start_time,end_time,user
0,1,New Client Onboarding Request,2017-11-28 18:00:00.000000,2017-11-28 18:32:37.799670,Tom
1,1,Review Documents,2017-11-28 18:37:37.799670,2017-11-28 18:48:08.319336,Freddy
2,1,Automated Scoreboarding,2017-11-28 18:53:08.319336,2017-11-28 18:59:32.546680,SYSTEM
3,1,Update Backend Systems,2017-11-28 19:04:32.546680,2017-11-28 19:13:41.754157,RPA
4,1,Notification Review Request Completed,2017-11-28 19:18:41.754157,2017-11-28 19:26:44.284743,SYSTEM
5,2,New Client Onboarding Request,2017-11-29 18:00:00.000000,2017-11-29 18:40:47.579016,Tom
6,2,Review Documents,2017-11-29 18:45:47.579016,2017-11-29 19:21:22.732782,Jane
7,2,Automated Scoreboarding,2017-11-29 19:26:22.732782,2017-11-29 19:33:29.563456,SYSTEM
8,2,Update Backend Systems,2017-11-29 19:38:29.563456,2017-11-29 19:53:31.751123,RPA
9,2,Notification Review Request Completed,2017-11-29 19:58:31.751123,2017-11-29 20:07:52.740973,SYSTEM


## Add Task Level Business Data : UserGroup

In [82]:
# Add a column for UserGroup
df["user_group"] = ""

In [83]:
def setUserGroup(row):
    match row["task"]:
        case "Review Documents":
            return "Operations"
        case "New Client Onboarding Request":
            return "Sales" 
        case "Manual Scoreboarding":
            return "Risk"
        case "Update Backend Systems":
            return "SYSTEM"
        case "Automated Scoreboarding":
            return "SYSTEM"
        case "Notification Review Request Completed":
            return "SYSTEM"
        case _:
            return ""

In [84]:
df["user_group"] = df.apply(setUserGroup, axis=1)

In [85]:
df.head(20)

,process_id,task,start_time,end_time,user,user_group
0,1,New Client Onboarding Request,2017-11-28 18:00:00.000000,2017-11-28 18:32:37.799670,Tom,Sales
1,1,Review Documents,2017-11-28 18:37:37.799670,2017-11-28 18:48:08.319336,Freddy,Operations
2,1,Automated Scoreboarding,2017-11-28 18:53:08.319336,2017-11-28 18:59:32.546680,SYSTEM,SYSTEM
3,1,Update Backend Systems,2017-11-28 19:04:32.546680,2017-11-28 19:13:41.754157,RPA,SYSTEM
4,1,Notification Review Request Completed,2017-11-28 19:18:41.754157,2017-11-28 19:26:44.284743,SYSTEM,SYSTEM
5,2,New Client Onboarding Request,2017-11-29 18:00:00.000000,2017-11-29 18:40:47.579016,Tom,Sales
6,2,Review Documents,2017-11-29 18:45:47.579016,2017-11-29 19:21:22.732782,Jane,Operations
7,2,Automated Scoreboarding,2017-11-29 19:26:22.732782,2017-11-29 19:33:29.563456,SYSTEM,SYSTEM
8,2,Update Backend Systems,2017-11-29 19:38:29.563456,2017-11-29 19:53:31.751123,RPA,SYSTEM
9,2,Notification Review Request Completed,2017-11-29 19:58:31.751123,2017-11-29 20:07:52.740973,SYSTEM,SYSTEM


## Add Process Instance Business Data : Industry
Industy won't change during the process so all tasks for a given process ID must have the same value

In [86]:
df["industry"] = ""

In [87]:
process_id_list = df["process_id"].unique()

for process_id in process_id_list:
    industry = random.choice(['Federal','Finance','Healthcare','Insurance','Telecom'])
    df.loc[df["process_id"].eq(process_id), "industry"] = industry
    
                          
    

In [88]:
df.head(20)

,process_id,task,start_time,end_time,user,user_group,industry
0,1,New Client Onboarding Request,2017-11-28 18:00:00.000000,2017-11-28 18:32:37.799670,Tom,Sales,Healthcare
1,1,Review Documents,2017-11-28 18:37:37.799670,2017-11-28 18:48:08.319336,Freddy,Operations,Healthcare
2,1,Automated Scoreboarding,2017-11-28 18:53:08.319336,2017-11-28 18:59:32.546680,SYSTEM,SYSTEM,Healthcare
3,1,Update Backend Systems,2017-11-28 19:04:32.546680,2017-11-28 19:13:41.754157,RPA,SYSTEM,Healthcare
4,1,Notification Review Request Completed,2017-11-28 19:18:41.754157,2017-11-28 19:26:44.284743,SYSTEM,SYSTEM,Healthcare
5,2,New Client Onboarding Request,2017-11-29 18:00:00.000000,2017-11-29 18:40:47.579016,Tom,Sales,Insurance
6,2,Review Documents,2017-11-29 18:45:47.579016,2017-11-29 19:21:22.732782,Jane,Operations,Insurance
7,2,Automated Scoreboarding,2017-11-29 19:26:22.732782,2017-11-29 19:33:29.563456,SYSTEM,SYSTEM,Insurance
8,2,Update Backend Systems,2017-11-29 19:38:29.563456,2017-11-29 19:53:31.751123,RPA,SYSTEM,Insurance
9,2,Notification Review Request Completed,2017-11-29 19:58:31.751123,2017-11-29 20:07:52.740973,SYSTEM,SYSTEM,Insurance


## Add Process Instance Business Data : Service Charge
Service charge will be a random choice based on industry.

In [89]:
df["service_charge"] = 0

In [90]:
def getServiceChargeByIndustry(industry):
    match industry:
        case "Federal":
            return random.choice([3000, 6000, 8000])
        case "Finance":
            return random.choice([10000, 12000, 20000])
        case "Healthcare":
            return random.choice([15000, 20000, 25000])
        case "Insurance":
            return 45000
        case "Telecom":
            return 49000
        case _:
            return 64000

In [91]:
last_process = -1
service_charge = 0

process_id_list = df["process_id"].unique()

for process_id in process_id_list:
        # get the tasks for this process instance, use copy() to make it clear to the interpreter we know
        # this is a copy to supress any warnings about updating copies
        process_instance_tasks = df.loc[df.process_id == process_id].copy()
        # Get the fir row out of the task list and get the industry value
        industry = process_instance_tasks.iloc[0]['industry']
        
        # call function to get a semi-random service charge for the industry
        service_charge = getServiceChargeByIndustry(industry)
        
        # set the service charge in every row for this process instance
        df.loc[df.process_id == process_id, 'service_charge'] = service_charge
    


In [92]:
df.head(50)

,process_id,task,start_time,end_time,user,user_group,industry,service_charge
0,1,New Client Onboarding Request,2017-11-28 18:00:00.000000,2017-11-28 18:32:37.799670,Tom,Sales,Healthcare,25000
1,1,Review Documents,2017-11-28 18:37:37.799670,2017-11-28 18:48:08.319336,Freddy,Operations,Healthcare,25000
2,1,Automated Scoreboarding,2017-11-28 18:53:08.319336,2017-11-28 18:59:32.546680,SYSTEM,SYSTEM,Healthcare,25000
3,1,Update Backend Systems,2017-11-28 19:04:32.546680,2017-11-28 19:13:41.754157,RPA,SYSTEM,Healthcare,25000
4,1,Notification Review Request Completed,2017-11-28 19:18:41.754157,2017-11-28 19:26:44.284743,SYSTEM,SYSTEM,Healthcare,25000
5,2,New Client Onboarding Request,2017-11-29 18:00:00.000000,2017-11-29 18:40:47.579016,Tom,Sales,Insurance,45000
6,2,Review Documents,2017-11-29 18:45:47.579016,2017-11-29 19:21:22.732782,Jane,Operations,Insurance,45000
7,2,Automated Scoreboarding,2017-11-29 19:26:22.732782,2017-11-29 19:33:29.563456,SYSTEM,SYSTEM,Insurance,45000
8,2,Update Backend Systems,2017-11-29 19:38:29.563456,2017-11-29 19:53:31.751123,RPA,SYSTEM,Insurance,45000
9,2,Notification Review Request Completed,2017-11-29 19:58:31.751123,2017-11-29 20:07:52.740973,SYSTEM,SYSTEM,Insurance,45000


# Stories
Make a copy of the dataframe so we can restart here if required.

In [136]:
story_df = df.copy()

## Story : 1 - Rod is Too Busy
Rod is always very busy doing other work, so he may not be able to start right away.
Lets take a look at two `Review Documents` tasks that have been done by Rod. 

In [137]:
filter = (story_df['task'] == 'Review Documents') & (story_df['user'] == 'Rod')
story_df[filter].head(2)

,process_id,task,start_time,end_time,user,user_group,industry,service_charge
21,5,Review Documents,2017-12-02 18:18:16.932222,2017-12-02 18:26:53.144292,Rod,Operations,Telecom,49000
41,9,Review Documents,2017-12-06 18:26:41.470554,2017-12-06 22:00:43.274137,Rod,Operations,Insurance,45000


Delay the start time of Rod's `Review Documents` tasks by 1 hour

In [138]:
# If user is Rod and activity is `Review Documents` shift the start_time by 1 hour
# 
story_df = shiftActivityStartTime(story_df, 'user','Rod', 'Review Documents', 1)

Lets see the result of this change:

In [139]:
filter = (story_df['task'] == 'Review Documents') & (story_df['user'] == 'Rod')
story_df[filter].head(2)

,process_id,task,start_time,end_time,user,user_group,industry,service_charge
21,5,Review Documents,2017-12-02 19:18:16.932222,2017-12-02 19:26:53.144292,Rod,Operations,Telecom,49000
41,9,Review Documents,2017-12-06 19:26:41.470554,2017-12-06 23:00:43.274137,Rod,Operations,Insurance,45000


## Story : 2 - Complex Doc Review
Increase the time taken for doc review by 2hrs when the industry is federal. 
The rationale here is that the documentation requirements are greater.

Lets look at some `Review Documents` where the industry is `Federal`

In [140]:
filter = (story_df['task'] == 'Review Documents') & (story_df['industry'] == 'Federal')
story_df[filter].head(2)

,process_id,task,start_time,end_time,user,user_group,industry,service_charge
36,8,Review Documents,2017-12-05 18:25:17.935635,2017-12-05 18:37:56.936885,Jane,Operations,Federal,3000
66,14,Review Documents,2017-12-11 18:32:46.380264,2017-12-11 19:12:17.673376,Freddy,Operations,Federal,6000


In [141]:
story_df[story_df['process_id'] == 8]

,process_id,task,start_time,end_time,user,user_group,industry,service_charge
35,8,New Client Onboarding Request,2017-12-05 18:00:00.000000,2017-12-05 18:20:17.935635,Tom,Sales,Federal,3000
36,8,Review Documents,2017-12-05 18:25:17.935635,2017-12-05 18:37:56.936885,Jane,Operations,Federal,3000
37,8,Automated Scoreboarding,2017-12-05 18:42:56.936885,2017-12-05 18:51:59.644540,SYSTEM,SYSTEM,Federal,3000
38,8,Update Backend Systems,2017-12-05 18:56:59.644540,2017-12-05 19:08:00.309301,RPA,SYSTEM,Federal,3000
39,8,Notification Review Request Completed,2017-12-05 19:13:00.309301,2017-12-05 19:18:45.986885,SYSTEM,SYSTEM,Federal,3000


In [142]:
story_df = increaseDuration(story_df, 'industry','Federal', 'Review Documents', 2)

In [143]:
story_df[story_df['process_id'] == 8]

,process_id,task,start_time,end_time,user,user_group,industry,service_charge
35,8,New Client Onboarding Request,2017-12-05 18:00:00.000000,2017-12-05 18:20:17.935635,Tom,Sales,Federal,3000
36,8,Review Documents,2017-12-05 18:25:17.935635,2017-12-05 20:37:56.936885,Jane,Operations,Federal,3000
37,8,Automated Scoreboarding,2017-12-05 20:42:56.936885,2017-12-05 20:51:59.644540,SYSTEM,SYSTEM,Federal,3000
38,8,Update Backend Systems,2017-12-05 20:56:59.644540,2017-12-05 21:08:00.309301,RPA,SYSTEM,Federal,3000
39,8,Notification Review Request Completed,2017-12-05 21:13:00.309301,2017-12-05 21:18:45.986885,SYSTEM,SYSTEM,Federal,3000


## Export Finished Dataset

In [144]:
story_df.to_csv('pm_tasks_1k.csv', index=False, date_format='%d-%m-%Y %H:%M:%S')